In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import acquire
import os
import env
from sklearn.preprocessing import MinMaxScaler
from scipy import stats

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns
from scipy import stats

# import splitting and imputing functions
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

# turn off pink boxes for demo
import warnings
warnings.filterwarnings("ignore")

# import our own acquire module
import env
import acquire
import wrangle

# imports for modeling:
# import Logistic regression
from sklearn.linear_model import LogisticRegression
# import K Nearest neighbors:
from sklearn.neighbors import KNeighborsClassifier
# import Decision Trees:
from sklearn.tree import DecisionTreeClassifier, export_text, plot_tree
# import Random Forest:
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import classification_report, confusion_matrix

# interpreting our models:
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


directory = os.getcwd()

In [2]:
def get_connection_url(db, user=env.user, host=env.host, password=env.password):
    """
    This function will:
    - take username, pswd, host credentials from imported env module
    - output a formatted connection_url to access mySQL db
    """
    return f'mysql+pymysql://{user}:{password}@{host}/{db}'

In [3]:
SQL_query = '''
SELECT bedroomcnt, 
bathroomcnt, 
calculatedfinishedsquarefeet, 
taxvaluedollarcnt, 
yearbuilt, 
taxamount, 
fips 
FROM properties_2017 
WHERE propertylandusetypeid  = 261
'''

In [4]:
def new_zillow_data(SQL_query):
    """
    This function will:
    - take in a SQL_query
    - create a connection_url to mySQL
    - return a df of the given query from the zillow db
    """
    url = get_connection_url('zillow')
    
    return pd.read_sql(SQL_query, url)

In [5]:
def get_zillow_data(SQL_query, directory, filename = 'zillow.csv'):
    """
    This function will:
    - Check local directory for csv file
        - return if exists
    - if csv doesn't exist:
        - creates df of sql query
        - writes df to csv
    - outputs zillow df
    """
    if os.path.exists(directory+filename): 
        df = pd.read_csv(filename)
        return df
    else:
        df = new_zillow_data(SQL_query)

        df.to_csv(filename)
        return df

**Let's get our data**

In [6]:
df = get_zillow_data(SQL_query, directory)
df.head()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
0,0.0,0.0,NaN,27516.0,NaN,NaN,6037.0
1,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
2,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
3,0.0,0.0,NaN,2108.0,NaN,174.21,6037.0
4,4.0,2.0,3633.0,296425.0,2005.0,6941.39,6037.0


**Now for some exploratory looks at the data**

In [7]:
df.shape

(2152863, 7)

In [8]:
df.info(show_counts = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2152863 entries, 0 to 2152862
Data columns (total 7 columns):
 #   Column                        Non-Null Count    Dtype  
---  ------                        --------------    -----  
 0   bedroomcnt                    2152852 non-null  float64
 1   bathroomcnt                   2152852 non-null  float64
 2   calculatedfinishedsquarefeet  2144379 non-null  float64
 3   taxvaluedollarcnt             2152370 non-null  float64
 4   yearbuilt                     2143526 non-null  float64
 5   taxamount                     2148421 non-null  float64
 6   fips                          2152863 non-null  float64
dtypes: float64(7)
memory usage: 115.0 MB


In [9]:
df.describe()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
count,2.152852e+06,2.152852e+06,2.144379e+06,2.152370e+06,2.143526e+06,2.148421e+06,2.152863e+06
mean,3.287196e+00,2.230688e+00,1.862855e+03,4.618962e+05,1.960950e+03,5.634866e+03,6.048377e+03
std,9.547544e-01,9.992796e-01,1.222125e+03,6.996760e+05,2.216220e+01,8.178910e+03,2.043329e+01
min,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.801000e+03,1.850000e+00,6.037000e+03
25%,3.000000e+00,2.000000e+00,1.257000e+03,1.881702e+05,1.949000e+03,2.534980e+03,6.037000e+03
50%,3.000000e+00,2.000000e+00,1.623000e+03,3.276710e+05,1.958000e+03,4.108950e+03,6.037000e+03
75%,4.000000e+00,3.000000e+00,2.208000e+03,5.345270e+05,1.976000e+03,6.414320e+03,6.059000e+03
max,2.500000e+01,3.200000e+01,9.525760e+05,9.842891e+07,2.016000e+03,1.337756e+06,6.111000e+03


**Do we have any nulls?**

In [10]:
df.isnull().sum() #Quite a bit of nulls

bedroomcnt                        11
bathroomcnt                       11
calculatedfinishedsquarefeet    8484
taxvaluedollarcnt                493
yearbuilt                       9337
taxamount                       4442
fips                               0
dtype: int64

In [11]:
df = df.dropna() # Because there are 2152863 rows, dropping them shouldn't hurt the data so much

In [12]:
df.info() #Looking at the data after dropping the nulls

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2140235 entries, 4 to 2152862
Data columns (total 7 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   bedroomcnt                    float64
 1   bathroomcnt                   float64
 2   calculatedfinishedsquarefeet  float64
 3   taxvaluedollarcnt             float64
 4   yearbuilt                     float64
 5   taxamount                     float64
 6   fips                          float64
dtypes: float64(7)
memory usage: 130.6 MB


In [13]:
df.isnull().sum() #Nulls Be Gone

bedroomcnt                      0
bathroomcnt                     0
calculatedfinishedsquarefeet    0
taxvaluedollarcnt               0
yearbuilt                       0
taxamount                       0
fips                            0
dtype: int64

In [14]:
df.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2140235 entries, 4 to 2152862
Data columns (total 7 columns):
 #   Column                        Non-Null Count    Dtype  
---  ------                        --------------    -----  
 0   bedroomcnt                    2140235 non-null  float64
 1   bathroomcnt                   2140235 non-null  float64
 2   calculatedfinishedsquarefeet  2140235 non-null  float64
 3   taxvaluedollarcnt             2140235 non-null  float64
 4   yearbuilt                     2140235 non-null  float64
 5   taxamount                     2140235 non-null  float64
 6   fips                          2140235 non-null  float64
dtypes: float64(7)
memory usage: 130.6 MB


In [15]:
df.bedroomcnt = df.bedroomcnt.astype(int) # This is to change the data in 'bedroomcnt' to a whole number

In [16]:
df.yearbuilt = df.yearbuilt.astype(int) # This is to change the data in 'yearbuilt' to a whole number

In [17]:
df.head() #Making sure that the above changes were implemented correctly

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
4,4,2.0,3633.0,296425.0,2005,6941.39,6037.0
6,3,4.0,1620.0,847770.0,2011,10244.94,6037.0
7,3,2.0,2077.0,646760.0,1926,7924.68,6037.0
11,0,0.0,1200.0,5328.0,1972,91.60,6037.0
14,0,0.0,171.0,6920.0,1973,255.17,6037.0


In [18]:
# I didn't like the names of the columns, so let's change them
df = df.rename(columns={"bedroomcnt": "Bedroom_Count"})
df = df.rename(columns={"bathroomcnt": "Bathroom_Count"})
df = df.rename(columns={"calculatedfinishedsquarefeet": "Finished_sqft"})
df = df.rename(columns={"taxvaluedollarcnt": "Tax_value_dollars"})
df = df.rename(columns={"yearbuilt": "Year_built"})
df = df.rename(columns={"taxamount": "Tax_amount"})
df = df.rename(columns={'fips': 'County'})
df.County = df.County.map({6037:'LA',6059:'Orange',6111:'Ventura'})
df.head(4)

,Bedroom_Count,Bathroom_Count,Finished_sqft,Tax_value_dollars,Year_built,Tax_amount,County
4,4,2.0,3633.0,296425.0,2005,6941.39,LA
6,3,4.0,1620.0,847770.0,2011,10244.94,LA
7,3,2.0,2077.0,646760.0,1926,7924.68,LA
11,0,0.0,1200.0,5328.0,1972,91.60,LA


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2140235 entries, 4 to 2152862
Data columns (total 7 columns):
 #   Column             Dtype  
---  ------             -----  
 0   Bedroom_Count      int64  
 1   Bathroom_Count     float64
 2   Finished_sqft      float64
 3   Tax_value_dollars  float64
 4   Year_built         int64  
 5   Tax_amount         float64
 6   County             object 
dtypes: float64(4), int64(2), object(1)
memory usage: 130.6+ MB


**Let's create a function to split the data**

In [20]:
def split_zillow_data(df):
    '''
    This function performs split on zillow data.
    Returns train, validate, and test dfs.
    '''
    train_validate, test = train_test_split(df, test_size=.2, 
                                        random_state=123)
                                        
    train, validate = train_test_split(train_validate, test_size=.3, 
                                   random_state=123)
    return train, validate, test

In [21]:
train, validate, test = split_zillow_data(df)

In [22]:
train.head() # Everything worked as planned

,Bedroom_Count,Bathroom_Count,Finished_sqft,Tax_value_dollars,Year_built,Tax_amount,County
1567873,3,1.0,1248.0,191578.0,1950,2712.21,LA
1218138,2,2.0,1326.0,1345206.0,1950,16539.04,LA
339661,2,1.0,1053.0,356648.0,1953,4575.16,LA
1017133,3,2.0,1256.0,175069.0,1946,2635.51,LA
40250,3,2.0,1640.0,543000.0,1957,6344.96,Orange


**Let's create a wrangle function to implement all changes I did above for future use**

In [29]:
def wrangle_zillow():
    SQL_query = '''
    SELECT 
        bedroomcnt, 
        bathroomcnt, 
        calculatedfinishedsquarefeet, 
        taxvaluedollarcnt, 
        yearbuilt, 
        taxamount, 
        fips 
    FROM 
        properties_2017 
    WHERE 
        propertylandusetypeid  = 261
    '''
    df = get_zillow_data(SQL_query, directory)
    df = df.dropna()
    df.bedroomcnt = df.bedroomcnt.astype(int)
    df.yearbuilt = df.yearbuilt.astype(int)
    df = df.rename(columns={"bedroomcnt": "Bedroom_Count"})
    df = df.rename(columns={"bathroomcnt": "Bathroom_Count"})
    df = df.rename(columns={"calculatedfinishedsquarefeet": "Finished_sqft"})
    df = df.rename(columns={"taxvaluedollarcnt": "Tax_value_dollars"})
    df = df.rename(columns={"yearbuilt": "Year_built"})
    df = df.rename(columns={"taxamount": "Tax_amount"})
    df = df.rename(columns={'fips': 'County'})
    df.County = df.County.map({6037:'LA',6059:'Orange',6111:'Ventura'})
    return df

**Let's test our function to make sure it worked**

In [30]:
df = wrangle_zillow()
df.head() # Looks good to me

,Bedroom_Count,Bathroom_Count,Finished_sqft,Tax_value_dollars,Year_built,Tax_amount,County
4,4,2.0,3633.0,296425.0,2005,6941.39,LA
6,3,4.0,1620.0,847770.0,2011,10244.94,LA
7,3,2.0,2077.0,646760.0,1926,7924.68,LA
11,0,0.0,1200.0,5328.0,1972,91.60,LA
14,0,0.0,171.0,6920.0,1973,255.17,LA


In [31]:
df.info(show_counts = True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2140235 entries, 4 to 2152862
Data columns (total 7 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   Bedroom_Count      2140235 non-null  int64  
 1   Bathroom_Count     2140235 non-null  float64
 2   Finished_sqft      2140235 non-null  float64
 3   Tax_value_dollars  2140235 non-null  float64
 4   Year_built         2140235 non-null  int64  
 5   Tax_amount         2140235 non-null  float64
 6   County             2140235 non-null  object 
dtypes: float64(4), int64(2), object(1)
memory usage: 130.6+ MB


In [32]:
df.isnull().sum()

Bedroom_Count        0
Bathroom_Count       0
Finished_sqft        0
Tax_value_dollars    0
Year_built           0
Tax_amount           0
County               0
dtype: int64

In [33]:
df.shape

(2140235, 7)

In [34]:
df.info(show_counts = True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2140235 entries, 4 to 2152862
Data columns (total 7 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   Bedroom_Count      2140235 non-null  int64  
 1   Bathroom_Count     2140235 non-null  float64
 2   Finished_sqft      2140235 non-null  float64
 3   Tax_value_dollars  2140235 non-null  float64
 4   Year_built         2140235 non-null  int64  
 5   Tax_amount         2140235 non-null  float64
 6   County             2140235 non-null  object 
dtypes: float64(4), int64(2), object(1)
memory usage: 130.6+ MB
